In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import geopandas as gpd

In [48]:
df = pd.read_csv('OdinData/odin-2018-2019-v1.0.0.csv')

In [49]:
# aim = ['vertpc', 'aankpc', 'khvm', 'weekdag', 'choice_dur', 'choice_dist', 'bike_dur', 'bike_dist', 'car_dur', 'car_dist', 'pt_dur', 'pt_dist', 'walk_dur', 'walk_dist', 'hour'

In [50]:
df.set_index('verplid', inplace = True)
df.drop(['wopc', 'wogem', 'sted', 'gemgr', 'prov', 'opid'], axis = 1, inplace = True)

In [51]:
# PCs = gpd.read_file('../PublicGeoJsons/AmsPCs.json')[['Postcode4', 'geometry']]
# merged = df.merge(PCs, left_on = 'vertpc', right_on = 'Postcode4')
# df = gpd.GeoDataFrame(
#     merged,
#     geometry= merged.geometry,
#     crs='EPSG:4326'
# )

In [52]:
# merged2 = df[['aankpc']].merge(PCs, left_on = 'aankpc', right_on = 'Postcode4')
# aank = gpd.GeoDataFrame(
#     merged2,
#     geometry= merged2.geometry,
#     crs='EPSG:4326'
# )

In [53]:
# df['start_lon'] = aank.geometry.centroid.x
# df['start_lat'] = aank.geometry.centroid.y
# df['end_lon'] = df.geometry.centroid.x
# df['end_lat'] = df.geometry.centroid.y
#
# df.drop(['aankpc', 'vertpc', 'geometry', 'Postcode4'], axis =1, inplace = True)

In [54]:
df.drop('aanktijd', axis =1, inplace = True)
inf = pd.to_datetime(df['verttijd'],format= '%H:%M' )
df['seconds'] = inf.dt.second + 60*(inf.dt.hour)

seconds_in_day = 24*60*60
# df['sin_day'] = np.sin(2*np.pi*df.weekdag/7)
# df['cos_day'] = np.cos(2*np.pi*df.weekdag/7)
df['sin_time'] = np.sin(2*np.pi*df.seconds/seconds_in_day)
df['cos_time'] = np.cos(2*np.pi*df.seconds/seconds_in_day)

# df.drop(['weekdag', 'seconds', 'verttijd'], axis = 1, inplace = True)
df.drop([ 'seconds', 'verttijd'], axis = 1, inplace = True)

In [55]:
categorical_columns = pd.read_json('OdinData/odin-col-dict.json').columns
non_categorical_columns = [x for x in df.columns if x not in categorical_columns]
non_categorical_columns

['choice_dur',
 'choice_dist',
 'bike_dur',
 'bike_dist',
 'car_dur',
 'car_dist',
 'pt_dur',
 'pt_dist',
 'walk_dur',
 'walk_dist',
 'sin_time',
 'cos_time']

In [56]:
df.isna().sum()
# df.dropna(inplace = True)
# df.fillna(0)

hhpers               0
hhsam                0
hhlft1               0
hhlft2               0
hhlft3               0
hhlft4               0
geslacht             0
leeftijd             0
herkomst             0
betwerk              0
onbbez               0
opleiding            0
hhgestinkg           0
oprijbewijsau        0
hhauto               0
brandstofpa1         0
brandstofepa1        0
brandstofpa2         0
brandstofepa2        0
hhefiets             0
ovstkaart            0
weekdag              0
feestdag             0
doel                 0
kmotiefv             0
vertpc               0
aankpc               0
khvm                 0
choice_dur           0
choice_dist          0
bike_dur          5654
bike_dist         5654
car_dur          12917
car_dist         12917
pt_dur           30014
pt_dist          30014
walk_dur         59916
walk_dist        59916
sin_time             0
cos_time             0
dtype: int64

In [57]:
df[df.pt_dist.isna()][['vertpc', 'aankpc', 'walk_dist', 'bike_dist', 'car_dist', 'pt_dist']]
# df['A'].fillna(df['B'], inplace=True)

,vertpc,aankpc,walk_dist,bike_dist,car_dist,pt_dist
verplid,,,,,,
5602658095301,1191,1106,7375.85,7413.49,10380.46,NaN
5602658752503,1441,1121,7600.31,7631.54,10385.84,NaN
5602656133007,1969,1935,9922.41,10600.68,12790.36,NaN
5602656133008,1935,1969,9922.41,10614.76,12522.10,NaN
5602659574002,1975,1974,4890.14,4939.30,5739.89,NaN
...,...,...,...,...,...,...
5909567172101,1188,1185,3949.00,3951.00,NaN,NaN
5909577786502,1951,1561,NaN,13296.00,15185.00,NaN
5912013465803,8043,7938,NaN,52820.00,58293.00,NaN


In [58]:
df['walk_dist'] = df['walk_dist'].fillna(df['bike_dist']) #High Confidence
df['walk_dur'] = df['walk_dur'].fillna(df['bike_dur']) #High Confidence


In [59]:
df['pt_dist'] = df['pt_dist'].fillna(df['bike_dist']) #This is QUESTIONABLE
df['car_dist'] = df['car_dist'].fillna(df['bike_dist']) #Medium ocnfidence
df['car_dur'] = df['car_dur'].fillna(df['bike_dur']) #Medium ocnfidence
df['pt_dur'] = df['pt_dur'].fillna(df['bike_dur']) #This is QUESTIONABLE

df = df.dropna()


In [60]:
cat_cols = list(set(categorical_columns).intersection(set(df.columns)))

In [61]:
df[cat_cols] = df[cat_cols].astype('object')

In [62]:
df.dtypes

hhpers            object
hhsam             object
hhlft1            object
hhlft2            object
hhlft3            object
hhlft4            object
geslacht          object
leeftijd          object
herkomst          object
betwerk           object
onbbez            object
opleiding         object
hhgestinkg        object
oprijbewijsau     object
hhauto            object
brandstofpa1      object
brandstofepa1     object
brandstofpa2      object
brandstofepa2     object
hhefiets          object
ovstkaart         object
weekdag           object
feestdag          object
doel              object
kmotiefv          object
vertpc            object
aankpc            object
khvm              object
choice_dur         int64
choice_dist        int64
bike_dur         float64
bike_dist        float64
car_dur          float64
car_dist         float64
pt_dur           float64
pt_dist          float64
walk_dur         float64
walk_dist        float64
sin_time         float64
cos_time         float64


In [63]:
import json

with open('OdinData/odin-col-dict.json', "r") as json_file:
    choice_dict = json.load(json_file)
df.khvm = df.khvm.astype(str)
df = df.replace({"khvm": choice_dict['khvm']})
df.weekdag = df.weekdag.astype(str)
df = df.replace({"weekdag": choice_dict['weekdag']})

In [64]:
df['khvm']

verplid
5602658095301               Bus/tram/metro
5602658752503                        Fiets
5602656133007    Personenauto - bestuurder
5602656133008    Personenauto - bestuurder
5602659574002    Personenauto - bestuurder
                           ...            
5912017839903                       Overig
5912017849203                        Fiets
5912018010402                        Fiets
5912018118903    Personenauto - bestuurder
5912018118904    Personenauto - bestuurder
Name: khvm, Length: 192618, dtype: object

In [65]:
df.to_pickle('Odin2019All')

In [70]:
df = pd.read_pickle('Odin2019All')

In [67]:
#2000-2037: Haarlem
DH = list(range(2490, 2599+ 1))
Rdam = list(range(3011,3089+ 1))
Adam = list(range(1011, 1109 + 1)) + list(range(1381, 1384 +1))
Eind = list(range(5611,5658+1)) + list(range(5660,5667+1))#Geldrop
Tilb = list(range(5011,5049+1))
Breda = list(range(4800-4839+1))
HaaBloZan = list(range(2000, 2061+1))
Ensc = list(range(7511, 7548+1))
Urban = DH + Adam + Rdam + Eind + Tilb + Breda + HaaBloZan+ Ensc

In [68]:
df = df[(df['aankpc'].isin(Urban)) | (df['vertpc'].isin(Urban))]
df.head()
df.to_pickle('Odin2019UrbanOr')
len(df)

50093

In [69]:
df = df[(df['aankpc'].isin(Urban)) & (df['vertpc'].isin(Urban))]
df.head()
df.to_pickle('Odin2019UrbanAnd')
len(df)

26287

In [71]:
df = df[(df['aankpc'].isin(Adam)) | (df['vertpc'].isin(Adam))]
df.to_pickle('Odin2019Ams')
df.head()
len(df)

14801

In [18]:
# Encode categorical features as numerical labels
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

In [17]:
# Split the data into train and test sets
X = df.drop('khvm', axis=1)  # Drop the target variable from the features
y = df['khvm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost classifier
clf = xgb.XGBClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Test the classifier
y_pred = clf.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

KeyboardInterrupt: 